Student 1: name: Larry Urevich, github:https://github.com/larryu04
Student 2: name: Nave Dan, github:https://github.com/NaveDan1
Student 3: name: Aviad Gabay, github:https://github.com/Avraham-Aviad-Gabay

Libraries (don't change)

In [ ]:
!pip -q install torchinfo mlflow openpyxl

from dataclasses import dataclass
from typing import List, Callable, Optional, Tuple
import itertools
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchinfo import summary

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support

import mlflow
import mlflow.pytorch

import matplotlib.pyplot as plt
%matplotlib inline


Device (don't change)

In [ ]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
use_amp = (DEVICE == "cuda")

print(f"Using device: {DEVICE}")
print(f"Mixed precision (AMP): {use_amp}")

Data (don't change)

In [ ]:

class DataManager:
    def __init__(self, dataset_class, root: str = "./data", val_fraction: float = 0.1,
                 batch_size: int = 32, seed: int = 42):
        self.dataset_class = dataset_class
        self.root = root
        self.val_fraction = val_fraction
        self.batch_size = batch_size
        self.seed = seed

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1918,), (0.3483,))
        ])

    def get_loaders(self) -> Tuple[DataLoader, DataLoader, DataLoader]:
        full_train = self.dataset_class(root=self.root, train=True,
                                        download=True, transform=self.transform)
        test_ds = self.dataset_class(root=self.root, train=False,
                                     download=True, transform=self.transform)

        val_size = int(len(full_train) * self.val_fraction)
        train_size = len(full_train) - val_size

        generator = torch.Generator().manual_seed(self.seed)
        train_ds, val_ds = random_split(full_train, [train_size, val_size], generator=generator)

        train_loader = DataLoader(train_ds, batch_size=self.batch_size,
                                  shuffle=True, num_workers=2, pin_memory=True)
        val_loader   = DataLoader(val_ds,   batch_size=self.batch_size,
                                  shuffle=False, num_workers=2, pin_memory=True)
        test_loader  = DataLoader(test_ds,  batch_size=self.batch_size,
                                  shuffle=False, num_workers=2, pin_memory=True)

        print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")
        return train_loader, val_loader, test_loader

Configurations (don't change)

In [ ]:

@dataclass
class LayerSpec:
    out_dim: int
    activation: Callable[[torch.Tensor], torch.Tensor] = F.relu
    dropout: float = 0.0
    batch_norm: bool = True
    weight_decay: float = 0.0

@dataclass
class ModelConfig:
    input_dim: Tuple[int, int, int] = (1, 28, 28)
    num_classes: int = 10
    layers: List[LayerSpec] = None

@dataclass
class TrainConfig:
    batch_size: int = 64
    epochs: int = 100
    lr: float = 1e-4
    patience: int = 15
    min_delta: float = 1e-4
    val_fraction: float = 0.1
    seed: int = 42


Model

In [ ]:

class MLPFromConfig(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        flat_dim = config.input_dim[0] * config.input_dim[1] * config.input_dim[2]
        self.layers_specs = config.layers
        layers = []
        prev_dim = flat_dim

        for i, spec in enumerate(config.layers):
            linear = nn.Linear(prev_dim, spec.out_dim)

            layers.append(linear)
            if spec.batch_norm:
                layers.append(nn.BatchNorm1d(spec.out_dim))
            if spec.dropout > 0:
                layers.append(nn.Dropout(spec.dropout))
            layers.append(spec.activation())
            prev_dim = spec.out_dim

        # Final classifier layer
        self.final_linear = nn.Linear(prev_dim, config.num_classes)
        layers.append(self.final_linear)

        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(x.size(0), -1)
        return self.net(x)

    def get_layer_params(self):
        param_groups = []
        for i, spec in enumerate(self.layers_specs):
            linear_layer = self.net[i * (4 if spec.batch_norm or spec.dropout > 0 else 3)]
            pass
        return self.layers_specs

Early Stopping (don't change)

In [ ]:

class EarlyStopping:
    def __init__(self, patience: int = 10, min_delta: float = 1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.should_stop = False

    def __call__(self, val_loss: float) -> bool:
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        return self.should_stop

Trainer (don't change)

In [ ]:

class Trainer:
    def __init__(self, model: nn.Module, config: TrainConfig):
        self.model = model.to(DEVICE)
        self.config = config
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = self._build_optimizer()
        self.scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
        self.early_stopping = EarlyStopping(patience=config.patience,
                                            min_delta=config.min_delta)

        self.history = {"train_loss": [], "train_acc": [],
                        "val_loss": [], "val_acc": []}

    def _build_optimizer(self):

        # Collect all Linear layers in the order they appear
        linear_layers = []
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Linear):
                linear_layers.append((name, module))

        param_groups = []

        for i, spec in enumerate(self.model.layers_specs):
            name, layer = linear_layers[i]
            param_groups.append({
                'params': layer.parameters(),
                'weight_decay': spec.weight_decay
            })

        final_name, final_layer = linear_layers[-1]
        param_groups.append({
            'params': final_layer.parameters(),
            'weight_decay': 0.0
        })

        return torch.optim.SGD(param_groups, momentum=0.9, nesterov=True, lr=self.config.lr)

    def _train_epoch(self, loader: DataLoader):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for data, target in loader:
            data, target = data.to(DEVICE), target.to(DEVICE)

            self.optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=use_amp):
                output = self.model(data)
                loss = self.criterion(output, target)

            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()

            total_loss += loss.item() * data.size(0)
            correct += (output.argmax(1) == target).sum().item()
            total += data.size(0)

        return total_loss / total, correct / total

    @torch.no_grad()
    def _eval_epoch(self, loader: DataLoader):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        for data, target in loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=use_amp):
                output = self.model(data)
                loss = self.criterion(output, target)

            total_loss += loss.item() * data.size(0)
            correct += (output.argmax(1) == target).sum().item()
            total += data.size(0)

        return total_loss / total, correct / total

    def fit(self, train_loader: DataLoader, val_loader: DataLoader):
        print("🚀 Starting training...\n")
        for epoch in range(1, self.config.epochs + 1):
            train_loss, train_acc = self._train_epoch(train_loader)
            val_loss, val_acc     = self._eval_epoch(val_loader)

            self.history["train_loss"].append(train_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_loss"].append(val_loss)
            self.history["val_acc"].append(val_acc)

            print(f"Epoch {epoch:3d} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            if self.early_stopping(val_loss):
                print(f"\n🛑 Early stopping triggered at epoch {epoch}")
                break

        print("\n✅ Training complete!")

    @torch.no_grad()
    def evaluate(self, loader: DataLoader):
        return self._eval_epoch(loader)

    @torch.no_grad()
    def predict_all(self, loader: DataLoader):
        self.model.eval()
        all_preds, all_targets = [], []
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            logits = self.model(x)
            preds = logits.argmax(dim=1).cpu().numpy()
            all_preds.append(preds)
            all_targets.append(y.numpy())
        return np.concatenate(all_preds), np.concatenate(all_targets)


    def save(self, path: str = "mlp_best.pt"):
        torch.save(self.model.state_dict(), path)
        print(f"💾 Model saved to {path}")



Run (do change)

In [ ]:


mlflow.set_experiment("KMNIST_Neural_Net_Optimization")

learning_rates = [1e-2,5e-3]
batch_sizes = [32, 64]
patience_vals = [5]

architecture_configs = [

    [
        LayerSpec(out_dim=512, activation=nn.ReLU, dropout=0.3, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.3, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.2, batch_norm=False, weight_decay=1e-3)
    ],

    [
        LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.5, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3)
    ],

    [
        LayerSpec(out_dim=512, activation=nn.ReLU, dropout=0.3, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.5, batch_norm=True, weight_decay=1e-3)
    ],

    [
         LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-2),
         LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.2, batch_norm=True, weight_decay=1e-2),
         LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.2, batch_norm=True, weight_decay=1e-2)
    ],

     [
        LayerSpec(out_dim=2048, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=1024, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=512, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
        LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3)
     ],

    [
         LayerSpec(out_dim=2048, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
         LayerSpec(out_dim=1024, activation=nn.ReLU, dropout=0.3, batch_norm=True, weight_decay=1e-3),
         LayerSpec(out_dim=512, activation=nn.ReLU, dropout=0.1, batch_norm=True, weight_decay=1e-3),
         LayerSpec(out_dim=256, activation=nn.ReLU, dropout=0.5, batch_norm=True, weight_decay=1e-3),
         LayerSpec(out_dim=128, activation=nn.ReLU, dropout=0.4, batch_norm=True, weight_decay=1e-3),
         LayerSpec(out_dim=64, activation=nn.ReLU, dropout=0.2, batch_norm=True, weight_decay=1e-3)

    ]
]

param_grid = list(itertools.product(learning_rates, batch_sizes, patience_vals, architecture_configs))

results_list = []

print(f"Total experiments to run: {len(param_grid)}")


for i, (lr, batch_size, patience, layers_config) in enumerate(param_grid):
    run_name = f"Run_{i+1}_LR_{lr}_BS_{batch_size}"
    print(f"\n--- Starting Experiment {i+1}/{len(param_grid)}: {run_name} ---")

    train_cfg = TrainConfig(
        batch_size=batch_size,
        epochs=50,
        lr=lr,
        patience=patience,
        val_fraction=0.1
    )

    model_cfg = ModelConfig(
        layers=layers_config
    )

    data_mgr = DataManager(
        dataset_class=datasets.KMNIST,
        val_fraction=train_cfg.val_fraction,
        batch_size=train_cfg.batch_size,
        seed=train_cfg.seed
    )
    train_loader, val_loader, test_loader = data_mgr.get_loaders()

    model = MLPFromConfig(model_cfg)

    with mlflow.start_run(run_name=run_name):

        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("patience", patience)
        mlflow.log_param("num_layers", len(layers_config))

        arch_desc = " -> ".join([str(l.out_dim) for l in layers_config])
        mlflow.log_param("architecture", arch_desc)

        trainer = Trainer(model, train_cfg)
        trainer.fit(train_loader, val_loader)


        test_preds, test_targets = trainer.predict_all(test_loader)
        test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_targets, test_preds, average='macro', zero_division=0)
        test_acc = accuracy_score(test_targets, test_preds)

        train_preds, train_targets = trainer.predict_all(train_loader)
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_targets, train_preds, average='macro', zero_division=0)
        train_acc = accuracy_score(train_targets, train_preds)

        mlflow.log_metric("test_accuracy", test_acc)
        mlflow.log_metric("test_f1", test_f1)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.log_metric("train_f1", train_f1)

        print(f"Result -> Test Acc: {test_acc:.4f}, Train Acc: {train_acc:.4f}")

        results_list.append({
            "experiment_id": i+1,
            "learning_rate": lr,
            "batch_size": batch_size,
            "patience": patience,
            "architecture": arch_desc,
            "train_accuracy": train_acc,
            "train_precision": train_precision,
            "train_recall": train_recall,
            "train_f1": train_f1,
            "test_accuracy": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f1": test_f1
        })

print("\nAll experiments completed.")

Visuazize the train


In [ ]:

history = trainer.history

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"],   label="Val Loss")
plt.title("Loss")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history["train_acc"], label="Train Acc")
plt.plot(history["val_acc"],   label="Val Acc")
plt.title("Accuracy")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()




Visualized the test (do change)

In [ ]:
# entire test result
test_loss, test_acc = trainer.evaluate(test_loader)
print(f"🏆 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# predictions
preds, targets = trainer.predict_all(test_loader)

# confusion matrix
cm = confusion_matrix(targets, preds)
plt.figure()
plt.imshow(cm)
plt.title("Confusion Matrix (Test)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.show()


# per-class report
print("Classification report (Test):")
print(classification_report(targets, preds, digits=4))



In [ ]:
from google.colab import files

df_results = pd.DataFrame(results_list)

print(df_results)

student_name = "Nave_Dan"
filename = f"{student_name}_ex2.xlsx"

df_results.to_excel(filename, index=False)
print(f"Saved results to {filename}")

files.download(filename)